# Translate eisenscript to eisenxml using lark

In [1]:
from pathlib import Path
from lark import Lark
from lark.visitors import Interpreter, Transformer, v_args
parser = Lark.open('eisenscript_02.lark')

In eisen tree structure I have nodes for
 - main
 - global_md
 - entry
 - call
 - loop
 - trans 
 - w_mod
 - md_mod

 will need an Interpreter method for each which adds something to the xml tree  
 how do I get token values?
 look   @visit_children_decor

In [2]:
# minimal example
test = """ 2 * { x 1 y 2} seed
{ rx 1 ry 2} seed
rule seed  { 
  box 
}
"""

In [3]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""

In [4]:
small = parser.parse(test)
small

Tree(Token('RULE', 'main'), [Tree(Token('RULE', 'entry'), [Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree('repeat', [Token('NUMBER', '2')]), Tree(Token('RULE', 'transgroup'), [Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'y'), Token('VALUE', '2')])])]), Tree('rulename', [Token('RULENAME', 'seed')])]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'transgroup'), [Tree(Token('RULE', 'trans'), [Token('TID', 'rx'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'ry'), Token('VALUE', '2')])])]), Tree('rulename', [Token('RULENAME', 'seed')])])]), Tree(Token('RULE', 'ruledef'), [Tree('rulename', [Token('RULENAME', 'seed')]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), []), Tree('rulename', [Token('RULENAME', 'box')])])])])

In [5]:
print(small.pretty())

main
  entry
    call
      loop
        repeat	2
        transgroup
          trans
            x
            1
          trans
            y
            2
      rulename	seed
    call
      loop
        transgroup
          trans
            rx
            1
          trans
            ry
            2
      rulename	seed
  ruledef
    rulename	seed
    call
      loop
      rulename	box



In [6]:
from xml.etree import ElementTree as et
from xml.dom import minidom


In [7]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = et.tostring(elem, "utf-8")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

class Itest(Interpreter):
    
    def convert(self, tree):
        self.xmltree = et.Element('rules')
        self.active = self.xmltree
        self._visit_tree(tree)
        return prettify(self.xmltree)
    
    def main(self, tree):
        self.visit_children(tree)

    def entry(self, tree):
        self.active = et.SubElement(self.xmltree, "rule")
        self.active.set("name", "entry")
        self.visit_children(tree)
        self.active = self.xmltree
        
    def call(self, tree):   
        # do all the xml for the call kids here
        # the call will consist of zero or more loops plus a rule name
        # extra nested loops will be converted to new rules and called in a chain
        

        # call = et.SubElement(self.active, "dummy tag")
        # current = self.active
        # self.loop_count = 0
        # self.active = call
        # print('len(call kids)', len(tree.children))
        data = self.visit_children(tree)
        rname = data[1]
        print(data)
        # if rname in ["box", "grid", "sphere", "line"]:
        #     call = et.SubElement(self.active, "instance")
        #     call.set("shape", rname)
        # else:
        #     call = et.SubElement(self.active, "call")
        #     call.set("rule", rname)
               

        # for loop in data[:-1]:
        #     if len(loop) == 2:
        #         count, tstr = loop
        #         print(count, tstr)
        #     elif len(loop) == 1:
        #         tstr = loop[-1]
        #         print(tstr)

        # can we make count = 1 rather than missing 
        # for loop in data[:-1]:
            



        # loop = data[0]
        # if len(loop) == 2:
        #     count, tstr = loop
        #     call.set("count", count)
        #     call.set("transforms", tstr)
        # elif len(loop) == 1:
        #     tstr = loop[-1]
        #     call.set("transforms", tstr)   
   
        # if len(data) > 2:
        #     # multiple loops
        #     # create sub rule for each extra loop
        #     for i, loop in enumerate(data[:-1]):
        #         newname = rname + "_{:02d}".format(i)
        #         call.set("rule", newname)





            # self.active.tag = "instance"
            # self.active.set("shape", rname)

        

        """_summary_
        `call  [[2, 'tx 1 ty 2'], 'seed']`
        `call  [['rx 1 ry 2'], 'seed']`
        `call [[10, 'tx 1'], [20, 'ty 2'], 'r1']`
        `call  [[], 'sphere']`
        """

    def transgroup(self, tree):
        # joins all the transforms inside {} into a string
        data = ' '.join(self.visit_children(tree))
        # print('transgroup ', data) 
        return data

    @v_args(meta=True)
    def trans(self, kids, meta):
        # called for each transform, returns a string
        tid = meta[0]
        values = meta[1:]
        
        if tid == "s" and len(values) == 1:
            tid = "sa"
        if tid in ("x", "y", "z"):
            tid = "t" + tid
        if tid in ('tx', 'ty', 'tz', 'rx', 'ry', 'rz', 'sa', 's'):
            tstr = tid + " " + " ".join(values) 
        else:
            tstr = ''
        # print('trans ', tstr)
        return tstr


    def loop(self, tree):
        # an optional count and the list of transforms in {} brackets 
        data = self.visit_children(tree)
        # print('loop ', data)
        # tstr = ' '.join(self.visit_children(tree))
        # print(tstr)
        # # self.active.set("transforms", tstr.strip())
        # self.loop_count += 1
        # print('loop count', self.loop_count)
        return data

    def ruledef(self, tree):
        # self.active = et.SubElement(self.xmltree, "rule")
        x = self.visit_children(tree)
        # print('ruledef ', x)
        # self.active = self.xmltree
        

            
    @v_args(inline=True)
    def rulename(self, rname):
        # can get here from ruledef or call or mmod
        # print('rulename ', rname)
        return str(rname)
        
        # if self.active.tag == "dummy tag":
        #     # parent is call
        #     if rname in ["box", "grid", "sphere", "line"]:
        #         self.active.tag = "instance"
        #         self.active.set("shape", rname)
        #     else: 
        #         self.active.tag = "call"
        #         self.active.set("rule", rname)
        # else:
        #     # parent is ruledef
        #     self.active.set("name", rname)
        

    @v_args(inline=True)
    def repeat(self, c):
        self.active.set("count", c)
        # print('repeat ', c)
        return int(c)

    @v_args(inline=True)
    def wmod(self, weight):
        self.active.set("weight", weight)

    @v_args(meta=True)
    def mmod(self, kids, meta):
        self.active.set("max_depth", meta[0])
        if len(meta) > 1:
            self.active.set("successor", meta[1])
      

In [8]:
it = Itest().convert(small)
print(it)

[[2, 'tx 1 ty 2'], 'seed']
[['rx 1 ry 2'], 'seed']
[[], 'box']
<?xml version="1.0" ?>
<rules>
  <rule name="entry" count="2"/>
</rules>




why do I get   
`loop  [Token('NUMBER', '2'), 'tx 1', 'ty 2']`
instead of 
`loop  ['2', 'tx 1', 'ty 2']`
as repeat returns 2, change to `return int(c)`

best structure for data returned from call's kids might be
`[2, 'tx 1 ty 2', 'seed' ]`

as of now have
`[[2, 'tx 1', 'ty 2'], 'seed']`
or 
`[['rx 1', 'ry 2'], 'seed']`
and for nested loops
`[[10, 'tx 1'], [20, 'ty 2'], 'r1']`
which will be hard to separate optional count.

now have
`call  [[2, 'tx 1 ty 2'], 'seed']`
`call  [['rx 1 ry 2'], 'seed']`
`call [[10, 'tx 1'], [20, 'ty 2'], 'r1']`
`call  [[], 'sphere']`
which will work


In [9]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 md 20  { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""
balltree = parser.parse(ball)
print(Itest().convert(balltree))

[[], 'R1']
[['tx 1 rz 3 ry 5'], 'R1']
[['s 1 1 0.1 '], 'box']
<?xml version="1.0" ?>
<rules weight="10" max_depth="20">
  <rule name="entry"/>
</rules>



can we make colour a generic string  
"red" 
           | "black"
           | "white"
           | "blue"
           | "green"

oct below has nested rules and currently fails to parse
change   
`call: count* loop* rn`  
to  
`call: (count* loop*)* rn`  

This only gets the last count * loop.  

My old code did this:

```xml
<?xml version="1.0" ?>
<rules max_depth="100">
  <rule name="entry">
    <call transforms="ry 36" count="10" rule="r1_00"/>
  </rule>
  <rule name="r1_00">
    <call transforms="ry 10" count="30" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <instance transforms="s 1 0.2 0.5" shape="box"/>
    <call transforms="sa 0.9 rz 5 rx 5 tx 1" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <instance transforms="s 1 0.2 0.5" shape="box"/>
    <call transforms="sa 0.99 rz -5 rx -5 tx 1" rule="r1"/>
  </rule>
</rules>
```

It's created an extra rule "r1_00" and dropped the third nested rule. The code does say it only deals with two loops per call.

Lark version visits all the loops but keeps overriding the atrributes
Creating the extra rules here could be done by adding to the tree?

Logic:  
if more than one loop per call:
  

item? - Zero or one instances of item (”maybe”)
item* - Zero or more instances of item
item+ - One or more instances of item
item ~ n - Exactly n instances of item
item ~ n..m - Between n to m instances of item (not recommended for wide ranges, due to performance issues)


In [10]:
oct = """set maxdepth 100

10 * { ry 36 sat 0.9 } 30 * { ry 10 } 1 * { h 30 b 0.8 sat 0.8 a 0.3  } r1

rule r1 w 20 {
   { s 0.9 rz 5 h 5 rx 5 x 1 }  r1
   { s 1 0.2 0.5 } box
}

rule r1 w 20 {
   {  s 0.99 rz -5 h 5 rx -5 x 1 }   r1
   { s 1 0.2 0.5 } box
}

rule r1  {
}"""

print(Itest().convert(parser.parse(oct)))

[[10, 'ry 36 '], [30], ['ry 10'], [1], ['   '], 'r1']
[['sa 0.9 rz 5  rx 5 tx 1'], 'r1']
[['s 1 0.2 0.5'], 'box']
[['sa 0.99 rz -5  rx -5 tx 1'], 'r1']
[['s 1 0.2 0.5'], 'box']
<?xml version="1.0" ?>
<rules weight="20">
  <rule name="entry" count="1"/>
</rules>



In [11]:
oct2 = """set maxdepth 100

10 * { x 1} 20 * { y 2 }  r1

rule r1  {
    {rx 1} box
    sphere
}"""

t = parser.parse(oct2)
print(Itest().convert(t))

"""
<?xml version="1.0" ?>
<rules max_depth="100">
  <rule name="entry">
    <call transforms="tx 1" count="10" rule="r1_00"/>
  </rule>
  <rule name="r1_00">
    <call transforms="ty 2" count="20" rule="r1"/>
  </rule>
  <rule name="r1">
    <instance transforms="" shape="box"/>
  </rule>
</rules>
"""

[[10, 'tx 1'], [20], ['ty 2'], 'r1']
[['rx 1'], 'box']
[[], 'sphere']
<?xml version="1.0" ?>
<rules>
  <rule name="entry" count="20"/>
</rules>



'\n<?xml version="1.0" ?>\n<rules max_depth="100">\n  <rule name="entry">\n    <call transforms="tx 1" count="10" rule="r1_00"/>\n  </rule>\n  <rule name="r1_00">\n    <call transforms="ty 2" count="20" rule="r1"/>\n  </rule>\n  <rule name="r1">\n    <instance transforms="" shape="box"/>\n  </rule>\n</rules>\n'

In [12]:
print(t.pretty())

main
  global_md	100
  entry
    call
      loop
        repeat	10
        transgroup
          trans
            x
            1
      loop
        repeat	20
      loop
        transgroup
          trans
            y
            2
      rulename	r1
  ruledef
    rulename	r1
    call
      loop
        transgroup
          trans
            rx
            1
      rulename	box
    call
      loop
      rulename	sphere



Logic for two loops:  
if 2 loops on call:  
    make another rule from 2nd loop  
    and call it from 1st loop  

logic for more loops  
    make a rule for each extra loop  
    call them in  a chain  

have in eisenscript_02.lark pushed the repeat down to the loop level 

All the bits for a new rule in one place.
Who should handle its creation? call or loop?

In [13]:
x = ['a']

In [14]:
len(x) == True

True